# Summary
This python script reads in a csv file downloaded from https://nces.ed.gov/collegenavigator/ and searches on Google for parking / transportation for the colleges, finding email contact info

- Thank you to https://www.pingshiuanchua.com/blog/post/scraping-search-results-from-google-search for providing helpful code related to web scraping

## Instructions

1. Download this Python notebook onto your computer (into folder of your choice)
2. Go to https://nces.ed.gov/collegenavigator/ and search for colleges / universities based on criteria (e.g., New York State) and click "Show Results"
3. Click "Export Results" and choose CSV to download the data
4. Delete any rows in the CSV file that you don't want contact info for (e.g., colleges with small campus setting). Make sure to delete rows at the end of the file that don't represent info for any colleges (e.g., "* Cohort Year ...") and save CSV file in the same folder as this Python notebook
5. Edit inputs / constants in the cell below
    - Ensure that COLLEGE_DATA_FILE has the name of the CSV file that was downloaded in the previous step
    - Note on current settings: code searches for "parking" or "transportation" department at each college and filters for only edu and org emails
    - There's generally no need to edit NUMBER_OF_RESULTS, USER_AGENT, and TIMEOUT_LIMIT
6. Run notebook and results will be saved in the same folder as this notebook when web scraping completes
7. Contact info is one line per website containing the email in the following format: email(s) (website where email was found);

In [1]:
"""
Section containing inputs / constants needed for code
"""

# File name of csv file downloaded from College Navigator
COLLEGE_DATA_FILE = 'CollegeNavigator_Search_2020-08-17_14.11.02.csv'

# File name of csv output after web scraping completes
OUTPUT_FILE = 'new_york_state_colleges.csv'

# Google search text before name of college
SEARCH_QUERY = '[parking OR transportation] department '

# Regular expression for emails
EMAIL_REGEX = r"[\w\.-]+@[\w-]+\.edu|[\w\.-]+@[\w-]+\.org"

# Number of google search results we want to return per search
NUMBER_OF_RESULTS = 10

# User agent needed to specify it is desktop search results that we want
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"

# Amount of seconds you are willing to spend opening a website / searching for email addresses
TIMEOUT_LIMIT = 10

In [2]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
# from fake_useragent import UserAgent
# ua = UserAgent()
import time
from multiprocessing import Process, Pool, TimeoutError

## Define functions for searching Google and finding email address

In [3]:
"""
Return all links found after conducting a Google search

Parameters
----------
query: String
    Google search query to be conducted
NUMBER_OF_RESULTS: int
    Number of search results that Google should return for the query
USER_AGENT:
    User agent for requests library (determines whether Google results are desktop v. mobile)

Returns
-------
links: List
    List of all links returned from Google search
"""


def search_google(query, NUMBER_OF_RESULTS, USER_AGENT):
    google_url = "https://www.google.com/search?q=" + \
        query + "&num=" + str(NUMBER_OF_RESULTS)
    google_request = requests.get(google_url, {"User-Agent": USER_AGENT})
    google_results_soup = BeautifulSoup(google_request.text, "html.parser")

    result_div = google_results_soup.find_all('div', attrs={'class': 'ZINbbc'})

    links = []
    for r in result_div:
        # Checks if each element is present, else, raise exception
        try:
            link = r.find('a', href=True)

            # Check to make sure everything is present before appending
            if link != '':
                links.append(link['href'])
        # Next loop if element is not present
        except:
            continue

    return links

In [4]:
"""
Cleans Google search results from search_google() function to just return websites

Parameters
----------
links: List
    List of all links returned from Google search
    
Returns
-------
clean_links: List
    List of all links returned from Google search (cleaned to only include web addresses)
"""


def clean_google_results(links):
    to_remove = []
    clean_links = []
    for i, l in enumerate(links):
        clean = re.search('\/url\?q\=(.*)\&sa', l)

        # Anything that doesn't fit the above pattern will be removed
        if clean is None:
            to_remove.append(i)
            continue
        clean_links.append(clean.group(1))

    print('  {0} google search result(s) found'.format(len(clean_links)))
    return clean_links

In [5]:
"""
Extracts all text elements from a website

Parameters
----------
web_address: String
    Website address to extract text from

Returns
-------
website_text: String
    All text elements from the website (concatenated together with commas as delimiters)
"""


def extract_text(web_address):
    # Download html data from web address
    website_html = requests.get(web_address, USER_AGENT)

    # Create beautiful soup object and find all text elements
    website_soup = BeautifulSoup(website_html.text, "html.parser")
    website_text_list = website_soup.find_all(text=True)

    # Convert list of text elements into string
    website_text = ', '.join(website_text_list)

    return website_text

In [6]:
"""
Extracts all email addresses from string

Parameters
----------
text: String
    Text to search for email addresses

Returns
-------
emails: Set
    Set containing all unique email addresses that were found in the text
"""


def extract_emails(text, EMAIL_REGEX):
    emails = set(re.findall(EMAIL_REGEX, text, re.I))

    return emails

## Read in CSV from College Navigator and search for parking / transportation emails

In [7]:
# Read in CSV containing college information
colleges = pd.read_csv(COLLEGE_DATA_FILE)
colleges.head()

,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID
0,Academy of Cosmetology and Esthetics NYC,"187 New Dorp Ln, Staten Island, New York 10306",www.theacademynyc.com,"< 2-year, Private for-profit",One but less than two years certificate,City: Large,No,NaN,NaN,84%,NaN,2016-17,"$17,253","Cosmetology/Cosmetologist, General",457855,4156000.0
1,Access Careers,"474 Fulton Avenue, Suite 201, Hempstead, New Y...",www.accesscareers.net,"< 2-year, Private for-profit",Less than one year certificate|One but less th...,Suburb: Large,No,NaN,NaN,87%,NaN,2016-17,NaN,"Practical Nursing, Vocational Nursing and Nurs...",483708,4213400.0
2,Access Careers-Islandia,"1930 Veterans Highway, Suite 10, Islandia, New...",www.accesscareers.net,"< 2-year, Private for-profit",Less than one year certificate|One but less th...,Suburb: Large,No,NaN,NaN,NaN,NaN,2016-17,NaN,"Practical Nursing, Vocational Nursing and Nurs...",494904,4213401.0
3,Ace Institute of Technology,"312 West 36 Street, 2nd Floor, New York, New Y...",www.acecareer.edu,"2-year, Private for-profit",One but less than two years certificate|Two bu...,City: Large,No,247.0,247.0,72%,NaN,Fall 2016,"$28,519",Information Science/Studies,449676,4043300.0
4,Adelphi University,"South Ave, Garden City, New York 11530-0701",www.adelphi.edu/,"4-year, Private not-for-profit",One but less than two years certificate|Associ...,Suburb: Large,Yes,7991.0,5360.0,73%,21%,Fall 2013,"$28,130",-,188429,266600.0


In [8]:
# Define the google search that needs to be conducted
colleges['google_search'] = SEARCH_QUERY + colleges['Name']
# Add empty column for contact info
colleges['contact_info_found'] = ''
colleges.head()

,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,google_search,contact_info_found
0,Academy of Cosmetology and Esthetics NYC,"187 New Dorp Ln, Staten Island, New York 10306",www.theacademynyc.com,"< 2-year, Private for-profit",One but less than two years certificate,City: Large,No,NaN,NaN,84%,NaN,2016-17,"$17,253","Cosmetology/Cosmetologist, General",457855,4156000.0,[parking OR transportation] department Academy...,
1,Access Careers,"474 Fulton Avenue, Suite 201, Hempstead, New Y...",www.accesscareers.net,"< 2-year, Private for-profit",Less than one year certificate|One but less th...,Suburb: Large,No,NaN,NaN,87%,NaN,2016-17,NaN,"Practical Nursing, Vocational Nursing and Nurs...",483708,4213400.0,[parking OR transportation] department Access ...,
2,Access Careers-Islandia,"1930 Veterans Highway, Suite 10, Islandia, New...",www.accesscareers.net,"< 2-year, Private for-profit",Less than one year certificate|One but less th...,Suburb: Large,No,NaN,NaN,NaN,NaN,2016-17,NaN,"Practical Nursing, Vocational Nursing and Nurs...",494904,4213401.0,[parking OR transportation] department Access ...,
3,Ace Institute of Technology,"312 West 36 Street, 2nd Floor, New York, New Y...",www.acecareer.edu,"2-year, Private for-profit",One but less than two years certificate|Two bu...,City: Large,No,247.0,247.0,72%,NaN,Fall 2016,"$28,519",Information Science/Studies,449676,4043300.0,[parking OR transportation] department Ace Ins...,
4,Adelphi University,"South Ave, Garden City, New York 11530-0701",www.adelphi.edu/,"4-year, Private not-for-profit",One but less than two years certificate|Associ...,Suburb: Large,Yes,7991.0,5360.0,73%,21%,Fall 2013,"$28,130",-,188429,266600.0,[parking OR transportation] department Adelphi...,


In [9]:
# Iterate through all colleges, searching google


for index, row in colleges.iterrows():
    print(
        'Searching on Google for parking / transportation emails for {0}'.format(row['Name']))
    
    
    query = row['google_search']
    links = search_google(query, NUMBER_OF_RESULTS, USER_AGENT)
    clean_links = clean_google_results(links)
    
    emails_list = ''
    
    # Iterate over clean links to find email addresses
    for link in clean_links:
        # Create a new pool which allows for timeouts
        pool1 = Pool(processes=1)
        
        # Try reading the text from websites
        try:
            link_text = pool1.apply_async(extract_text, (link,)).get(timeout=TIMEOUT_LIMIT)
            pool1.close()
            
            # Create another pool which allows for timeouts
            pool2 = Pool(processes = 1)
            
            # Try to extract emails from text
            try:
                email_set = pool2.apply_async(extract_emails, (link_text, EMAIL_REGEX)).get(timeout=TIMEOUT_LIMIT)
                pool2.close()
            
            # If there's a timeout error, just return an empty email set
            except TimeoutError:
                pool2.terminate()
                print('    Took too long to search text for emails in {0}'.format(link))
                email_set = {}
            
            # Calculate how many emails were found
            emails_found = len(email_set)
            print('    Found {0} email(s) at {1}'.format(emails_found, link))

            # If emails were found, add to list
            if emails_found > 0:
                emails_list += ', '.join(email_set) + ' (' + link + ');\n'
        
        # If there are any exceptions, skip and move on to the next link
        except:
            pool1.terminate()
            print('    Took too long / error with loading {0}'.format(link))
            continue
    
    # Add any emails that were found to the dataframe
    colleges.at[index, 'contact_info_found'] = emails_list.strip()
    print()

print('Scraping complete')

Searching on Google for parking / transportation emails for Academy of Cosmetology and Esthetics NYC
  10 google search result(s) found
    Found 0 email(s) at https://www.niche.com/colleges/academy-of-cosmetology--and--esthetics-nyc/
    Found 0 email(s) at https://www.internationalstudent.com/school-search/3873/usa/new-york/academy-of-cosmetology-and-esthetics-nyc/
    Found 0 email(s) at https://www.beautyschoolsdirectory.com/faq/cosmetology-school.php
    Found 0 email(s) at https://www.nytimes.com/2018/12/26/business/cosmetology-school-debt-iowa.html
    Took too long / error with loading https://www2.ed.gov/offices/OSFAP/PEPS/docs/closedschoolsearch.xls
    Found 0 email(s) at https://paulmitchell.edu/statenisland/programs/cosmetology
    Found 0 email(s) at https://paulmitchell.edu/statenisland
    Found 1 email(s) at https://www.empire.edu/consumer-information/international-students
    Found 0 email(s) at http://thesalonprofessionalacademy.edu/wp-content/uploads/2014/10/Studen

    Found 0 email(s) at https://www.jacobipeds.org/contact-us/map/
    Found 0 email(s) at https://en.parkopedia.com/parking/school/albert-einstein-college-of-medicine-of-yeshiva-university-ny/
    Found 0 email(s) at https://jacobirad.net/

Searching on Google for parking / transportation emails for Allen School-Brooklyn
  10 google search result(s) found
    Found 0 email(s) at https://www.allenschool.edu/brooklyn-campus/
    Found 0 email(s) at https://www.allenschool.edu/
    Found 0 email(s) at https://www.allenschool.edu/contact/
    Found 0 email(s) at https://www.schools.nyc.gov/schools/K306
    Found 1 email(s) at https://www.allenisd.org/domain/41
    Found 0 email(s) at https://en.wikipedia.org/wiki/Brooklyn_Technical_High_School
    Found 0 email(s) at https://www.glassdoor.com/Reviews/Allen-School-of-Health-Sciences-Reviews-E460788.htm
    Found 0 email(s) at https://www.allen-stevenson.org/
    Found 0 email(s) at https://www.schoolbusfleet.com/10009531/9-tips-for-a-smoot

    Found 12 email(s) at https://www.hunterbusinessschool.edu/wp-content/uploads/2018/07/Practical-Nursing-Student-Handbook.pdf
    Found 0 email(s) at https://www.practicalnursing.org/cost-lpn-program-school
    Found 0 email(s) at https://www.yelp.com/biz/amg-school-of-licensed-practical-nursing-new-york
    Found 3 email(s) at https://www.esboces.org/practicalnursing
    Found 0 email(s) at https://audaciafoundation.org/health-worker-retraining-initiative/

Searching on Google for parking / transportation emails for Apex Technical School
  10 google search result(s) found
    Found 0 email(s) at https://apexschool.com/
    Found 0 email(s) at https://apexschool.com/campus-old/
    Found 0 email(s) at https://www.parkwhiz.com/apex-technical-school-parking/
    Found 0 email(s) at https://www.yelp.com/biz/apex-technical-school-long-island-city
    Found 0 email(s) at https://www.campusexplorer.com/colleges/5A8E758F/New-York/Long-Island-City/Apex-Technical-School/
    Found 0 email(s) 

    Found 0 email(s) at https://www.romimu.com/elyon-teenage-camp-bochrim-8th-grade/

Searching on Google for parking / transportation emails for Bank Street College of Education
  10 google search result(s) found
    Found 1 email(s) at https://school.bankstreet.edu/admissions/transportation/
    Found 1 email(s) at https://school.bankstreet.edu/summer-camp/transportation-and-bus-service/
    Found 1 email(s) at https://www.bankstreet.edu/about-bank-street/offices-departments/human-resources-2/benefits/
    Found 0 email(s) at https://www.bankstreet.edu/
    Found 0 email(s) at https://en.parkopedia.com/parking/school/bank-street-college-of-education-ny/
    Found 0 email(s) at https://www.rand.org/pubs/research_briefs/RB9059.html
    Took too long / error with loading https://s3.amazonaws.com/bankstreet-wordpress/wp-content/uploads/sites/2/2018/04/03134302/GS-Catalog-2014-2015.pdf
    Found 0 email(s) at https://www.glassdoor.com/Salary/Bank-Street-College-of-Education-Salaries-E1285

  10 google search result(s) found
    Found 0 email(s) at https://barnard.edu/parking
    Found 0 email(s) at https://barnard.edu/admissions/parking
    Found 0 email(s) at https://barnard.edu/shuttle-services
    Found 4 email(s) at https://transportation.columbia.edu/content/im-looking-parking
    Found 2 email(s) at https://transportation.columbia.edu/content/247-parking-eligibility
    Found 0 email(s) at https://www.spotangels.com/nyc/barnard-college---columbia-university-parking
    Found 0 email(s) at https://spothero.com/nyc/barnard-college-parking
    Found 0 email(s) at https://www.statecollegepa.us/3059/Parking
    Found 2 email(s) at http://henrybarnardschool.org/our-school/parking-and-transportation/
    Found 0 email(s) at https://www.linkedin.com/in/elizabeth-wallach

Searching on Google for parking / transportation emails for Be'er Yaakov Talmudic Seminary
  10 google search result(s) found
    Found 0 email(s) at https://www.niche.com/colleges/beer-yaakov-talmudic-sem

...

/opt/anaconda3/lib/python3.7/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown status keyword 'n' in marked section
  warnings.warn(msg)


    Took too long / error with loading https://www.nyp.org/pdf/weillcornell_during.pdf
    Found 0 email(s) at https://en.parkopedia.com/parking/underground/cornell_university_weill_greenberg_center/10021/new_york/
    Found 0 email(s) at https://www.bestparking.com/new-york-ny-parking/destinations/new-york-presbyterian-weill-cornell-medical-center-parking/
    Found 0 email(s) at https://ctsurgery.weillcornell.org/patients-and-visitors/maps-directions-and-parking
    Found 0 email(s) at https://parking.com/nyc/destinations/weill-cornell-medical-college
    Found 0 email(s) at https://weillcornellbrainandspine.org/map-and-directions
    Found 0 email(s) at https://www.architectmagazine.com/design/buildings/weill-greenberg-center-at-weill-cornell-medical-college-new-york_o

Searching on Google for parking / transportation emails for Westchester School for Dental Assistant
  9 google search result(s) found
    Found 0 email(s) at https://dentaltrainingwp.com/
    Found 0 email(s) at http

  9 google search result(s) found
    Found 0 email(s) at https://www.niche.com/colleges/yeshiva-gedolah-kesser-torah/
    Found 0 email(s) at https://www.cappex.com/colleges/yeshiva-gedolah-kesser-torah
    Found 0 email(s) at https://www.collegefactual.com/colleges/yeshiva-gedolah-kesser-torah/paying-for-college/tuition-and-fees/
    Found 0 email(s) at https://www.youtube.com/watch%3Fv%3DvaEZdjoXPKU
    Found 0 email(s) at https://yeshivagedolahkessertorah.com/
    Found 0 email(s) at http://www.buzzfile.com/business/Yeshiva-Gedolah-Kesser-Torah-845-406-4308
    Found 0 email(s) at https://college-ranks.com/theology-religious-vocation/NY/Monsey/
    Found 0 email(s) at https://datausa.io/profile/geo/monsey-ny
    Found 0 email(s) at http://www.kessertorah.org/

Searching on Google for parking / transportation emails for Yeshiva Karlin Stolin
  10 google search result(s) found
    Found 0 email(s) at https://www.schools.nyc.gov/schools/KANX
    Found 0 email(s) at https://bigfuture.c

    Found 0 email(s) at https://patch.com/new-york/newcity/fire-captain-says-local-yeshivas-unsafe-video
    Found 0 email(s) at https://www.compass.com/listing/144-blauvelt-road-unit-120-monsey-ny-10952/373851000700964369/
    Found 0 email(s) at https://www.compass.com/listing/172-adar-court-monsey-ny-10952/165324847973401921/
    Found 0 email(s) at https://www.apartmentguide.com/apartments/New-York/Monsey/Regency-Village/100027993/
    Took too long / error with loading https://www.nysenate.gov/newsroom/press-releases/patrick-m-gallivan/votes-nys-senate-june-13-2019
    Found 0 email(s) at https://www.yumuseum.org/about/faq

Searching on Google for parking / transportation emails for Yeshiva Shaar Hatorah
  10 google search result(s) found
    Found 0 email(s) at https://www.oszlist.com/college/yeshiva-shaar-hatorah/
    Found 0 email(s) at https://www.neighborhoodscout.com/ny/queens/yeshiva-shaar-hatorah
    Found 0 email(s) at https://www.apartmentguide.com/apartments/New-York/Ye

    Found 0 email(s) at https://opecode.com/colleges/NY/%3Fpage%3D23
    Found 0 email(s) at https://opecode.com/theology-religious-vocation/%3Fpage%3D27

Searching on Google for parking / transportation emails for Yeshivas Maharit D'Satmar
  10 google search result(s) found
    Found 0 email(s) at http://yeshivasmaharit.org/
    Found 0 email(s) at http://yeshivasmaharit.org/wp-content/uploads/publications/Catalog-14-15.pdf
    Found 0 email(s) at https://www.cappex.com/colleges/yeshivas-maharit-dsatmar
    Found 0 email(s) at https://www.jta.org/2020/01/13/united-states/new-york-is-trying-to-reform-the-orthodox-yeshiva-system-which-some-graduates-say-barely-taught-them-to-speak-english
    Found 1 email(s) at https://collegescorecard.ed.gov/school/%3F488101-Yeshivas_Maharit_D%2527Satmar
    Found 0 email(s) at http://newhavenskiclub.com/813n8/the-satmar-world.html
    Found 0 email(s) at http://ludas.be/dubshhp/satmar-website.html
    Found 0 email(s) at https://www.medicalcareersdir

In [10]:
# Output results to CSV
colleges.to_csv(OUTPUT_FILE, index=False)
print('Saved data to csv')

Saved data to csv
